# P1. Exploration

## 1. Description Générale

En utilisant DBeaver et les paramètres de connexion donnés par ailleurs, faire une description détaillée de la base de données fournie. Présenter notamment le diagramme entité - relation (ERD) et les différentes caractéristiques des tables de la base.  Quelles types de renseignement peut-on tirer de cette base de données ?   

![erd](IMG/ERD_diagram.PNG)

Explications ERD : 


---

## 2. Exploration SQL via python / pandas
Dans cette partie, on s'attache à explorer la base de données en utilisant le language SQL pour les requêtes et le language Python (+ module pandas) pour récupérer les résultats de façon programmatique.  
Vous pouvez utiliser DBeaver ou outil similaire pour préparer / tester vos requêtes mais celles-ci doivent être reportées et appliquées dans du code Python comme présenté dans les quelques examples à suivre.  
*Pour les résultats "longs", un aperçu des quelques premières lignes et le comptage total des résultats est suffisant.*

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [4]:
print(conn)

### Organigramme Compagnie

Q01. Ecrivez et exécutez une unique requête listant chaque employée de la compagnie (Id, nom + prénom, Titre) ainsi que son manager direct (nom + prénom, Titre)

In [5]:
# requête de départ
query = """
    SELECT e.EmployeeId AS 'Id', e.firstname || ' ' || e.lastname AS 'Employee',
    e.Title AS "Employee title", m.firstname  || ' ' ||  m.lastname AS 'Manager',
    m.Title AS "Manager Title"   
    FROM employees e
    LEFT JOIN employees m ON e.ReportsTo = m.employeeId ;
"""

In [6]:
results = pd.read_sql_query(query, conn)
results

,Id,Employee,Employee title,Manager,Manager Title
0,1,Andrew Adams,General Manager,None,None
1,2,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
2,3,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,4,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,5,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,6,Michael Mitchell,IT Manager,Andrew Adams,General Manager
6,7,Robert King,IT Staff,Michael Mitchell,IT Manager
7,8,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


Q02. Pour chaque artiste présent dans cette BDD, indiquez le nombre d'albums de cet artiste également répertorié (Liste triée par nom d'artiste)


In [7]:
query = """SELECT a.Name AS Artist, count(*) AS nb_albums FROM artists a LEFT JOIN albums a2 ON a2.ArtistId = a.ArtistId 
GROUP BY a.Name 
ORDER BY a.Name ASC"""
results = pd.read_sql_query(query,conn)
results

,Artist,nb_albums
0,A Cor Do Som,1
1,AC/DC,2
2,Aaron Copland & London Symphony Orchestra,1
3,Aaron Goldberg,1
4,Academy of St. Martin in the Fields & Sir Nevi...,1
...,...,...
270,Xis,1
271,Yehudi Menuhin,1
272,Yo-Yo Ma,1
273,Youssou N'Dour,1


Q03. Produire une liste des albums reprenant également l'artiste de l'album, le nombre de pistes, la durée de toutes les pistes (en minutes) et le coût total (Liste triée par artiste puis nom d'album)


In [8]:
query = """SELECT a3.Name as Artist, a2.Title, count(TrackID) as Nb_tracks, SUM(Milliseconds/(1000*60)) as Minutes, SUM(UnitPrice) as Price 
FROM tracks t INNER JOIN albums a2 ON t.AlbumId = a2.AlbumId INNER JOIN artists a3 ON a2.ArtistId = a3.ArtistId 
GROUP BY t.AlbumId ORDER BY Artist, a2.Title DESC
"""
results = pd.read_sql_query(query,conn)
results

,Artist,Title,Nb_tracks,Minutes,Price
0,AC/DC,Let There Be Rock,8,38,7.92
1,AC/DC,For Those About To Rock We Salute You,10,35,9.90
2,Aaron Copland & London Symphony Orchestra,"A Copland Celebration, Vol. I",1,3,0.99
3,Aaron Goldberg,Worlds,1,4,0.99
4,Academy of St. Martin in the Fields & Sir Nevi...,The World of Classical Favourites,2,7,1.98
...,...,...,...,...,...
342,Vinícius De Moraes,Vinicius De Moraes,15,46,14.85
343,Wilhelm Kempff,Bach: Goldberg Variations,1,2,0.99
344,Yehudi Menuhin,Bartok: Violin & Viola Concertos,1,4,0.99
345,Yo-Yo Ma,Bach: The Cello Suites,1,2,0.99


Q04. Produire une liste des toutes les playlists reprenant son nom, le nombre de pistes, la durée de toutes les pistes (en minutes) et le coût total (Liste triée par nom de playlist)


In [9]:
query = """SELECT p.PlaylistId as Playlist_ID, p.Name as Name, COUNT(pt.TrackId) as Nb_tracks, COALESCE(SUM(Milliseconds/(1000*60)),0) as Minutes, COALESCE(SUM(UnitPrice),0) as Price
FROM playlists p LEFT JOIN playlist_track pt ON p.PlaylistId = pt.PlaylistId LEFT JOIN tracks t ON pt.TrackId = t.TrackId 
GROUP BY p.PlaylistId 
ORDER BY p.Name ASC"""
results = pd.read_sql_query(query,conn)
results

,Playlist_ID,Name,Nb_tracks,Minutes,Price
0,5,90’s Music,1477,5914,1462.23
1,4,Audiobooks,0,0,0.00
2,6,Audiobooks,0,0,0.00
3,11,Brazilian Music,39,141,38.61
4,12,Classical,75,329,74.25
5,13,Classical 101 - Deep Cuts,25,99,24.75
6,14,Classical 101 - Next Steps,25,117,24.75
7,15,Classical 101 - The Basics,25,113,24.75
8,16,Grunge,15,62,14.85
9,17,Heavy Metal Classic,26,123,25.74


Q05. Produire une liste des tous les genres de musique reprenant son genre, le nombre de pistes, la durée de toutes les pistes (en minutes), le coût total et le coût moyen (Liste triée par nom de genre)


In [10]:
query = """SELECT g.Name, COUNT(TrackId) as Nb_tracks, SUM(Milliseconds/(1000*60)) as Minutes, SUM(UnitPrice) as Price
FROM genres g INNER JOIN tracks t ON g.GenreId = t.GenreId 
GROUP BY g.GenreId 
ORDER BY g.Name ASC"""
results = pd.read_sql_query(query,conn)
results

,Name,Nb_tracks,Minutes,Price
0,Alternative,40,154,39.60
1,Alternative & Punk,332,1133,328.68
2,Blues,81,325,80.19
3,Bossa Nova,15,46,14.85
4,Classical,74,329,73.26
5,Comedy,17,442,33.83
6,Drama,64,2711,127.36
7,Easy Listening,24,64,23.76
8,Electronica/Dance,30,136,29.70
9,Heavy Metal,28,126,27.72


Q06. Indiquez les caractéristiques principales de toutes les pistes correspondant au genre 'Science Fiction' (nom de piste, album, artiste de l'album, compositeur, nom du type de media, durée (en secondes) et prix unitaire


In [11]:
query = """SELECT t.Name AS "Track name", a.Title AS Album, a2.Name AS Artist, t.Composer AS Composer, mt.Name as Media, Milliseconds/1000 as Seconds, t.UnitPrice as Price
FROM genres g 
INNER JOIN tracks t ON g.GenreId = t.GenreId 
INNER JOIN albums a ON t.AlbumId = a.AlbumId 
INNER JOIN artists a2 ON a.ArtistId = a2.ArtistId 
INNER JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId 
WHERE g.Name = "Science Fiction"
ORDER BY t.Name ASC"""
results = pd.read_sql_query(query,conn)
results

,Track name,Album,Artist,Composer,Media,Seconds,Price
0,A Day In the Life,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2620,1.99
1,A Measure of Salvation,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2563,1.99
2,Battlestar Galactica: The Story So Far,Battlestar Galactica: The Story So Far,Battlestar Galactica,None,Protected MPEG-4 video file,2622,1.99
3,Dirty Hands,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2627,1.99
4,Hero,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2713,1.99
5,Maelstrom,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2622,1.99
6,Rapture,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2624,1.99
7,Taking a Break from All Your Worries,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2624,1.99
8,The Eye of Jupiter,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2618,1.99
9,The Passage,"Battlestar Galactica, Season 3",Battlestar Galactica,None,Protected MPEG-4 video file,2623,1.99


Q07. Retrouvez tous les clients (nom, prénom, email, pays) ayant acheté de la musique de type 'World'


In [12]:
query = """SELECT DISTINCT c.FirstName, c.LastName, c.Email, c.Country 
FROM customers c 
INNER JOIN invoices i ON c.CustomerId = i.CustomerId 
INNER JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId 
INNER JOIN tracks t ON ii.TrackId = t.TrackId 
INNER JOIN genres g ON t.GenreId = g.GenreId 
WHERE g.Name = "World"
ORDER BY c.LastName """
results = pd.read_sql_query(query,conn)
results

,FirstName,LastName,Email,Country
0,Roberto,Almeida,roberto.almeida@riotur.gov.br,Brazil
1,João,Fernandes,jfernandes@yahoo.pt,Portugal
2,Bjørn,Hansen,bjorn.hansen@yahoo.no,Norway
3,Joakim,Johansson,joakim.johansson@yahoo.se,Sweden
4,Aaron,Mitchell,aaronmitchell@yahoo.ca,Canada
5,Steve,Murray,steve.murray@yahoo.uk,United Kingdom
6,Jennifer,Peterson,jenniferp@rogers.ca,Canada
7,Mark,Philips,mphilips12@shaw.ca,Canada
8,Martha,Silk,marthasilk@gmail.com,Canada


Q08. Nommez les 15 artistes comptant le plus de pistes de type 'Rock'
A REVOIR !!!!!!!!!!!!!!!!!!!!!!

In [13]:
query = """SELECT a.Name, COUNT(t.TrackId) as Nb_Rock_tracks 
FROM artists a 
INNER JOIN albums a2 ON a.ArtistId = a2.ArtistId 
INNER JOIN tracks t ON a2.AlbumId = t.AlbumId 
INNER JOIN genres g ON t.GenreId = g.GenreId 
WHERE g.Name = "Rock"
GROUP BY a.ArtistId 
ORDER BY Nb_Rock_tracks DESC
LIMIT 15"""
results = pd.read_sql_query(query,conn)
results

,Name,Nb_Rock_tracks
0,Led Zeppelin,114
1,U2,112
2,Deep Purple,92
3,Iron Maiden,81
4,Pearl Jam,54
5,Van Halen,52
6,Queen,45
7,The Rolling Stones,41
8,Creedence Clearwater Revival,40
9,Kiss,35


Q09. Nommez les 5 meilleurs clients 

In [14]:
query = """SELECT c.FirstName || ' ' || c.LastName AS Customer, SUM(i.Total) AS Total 
FROM invoices i INNER JOIN customers c ON i.CustomerId = c.CustomerId 
GROUP BY c.CustomerID
ORDER BY Total DESC 
LIMIT 5"""
results = pd.read_sql_query(query,conn)
results

,Customer,Total
0,Helena Holý,49.62
1,Richard Cunningham,47.62
2,Luis Rojas,46.62
3,Ladislav Kovács,45.62
4,Hugh O'Reilly,45.62


Q10. Détaillez (piste, album, artiste, genre et type media) tous les achats effectués par le client 'Richard Cunningham' (triée par date achat)


In [15]:
query = """SELECT i.InvoiceDate, t.Name AS Track, a.Title AS Album, a2.Name AS Artist, g.Name AS Genre, mt.Name AS Media
FROM customers c 
INNER JOIN invoices i ON c.CustomerId = i.CustomerId 
INNER JOIN invoice_items ii ON i.InvoiceId = ii.InvoiceId 
INNER JOIN tracks t ON ii.TrackId = t.TrackId 
INNER JOIN albums a ON t.AlbumId = a.AlbumId 
INNER JOIN artists a2 ON a.ArtistId = a2.ArtistId 
INNER JOIN genres g ON t.GenreId = g.GenreId 
INNER JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId 
WHERE c.FirstName = "Richard" AND c.LastName = "Cunningham"
ORDER BY i.InvoiceDate DESC"""
results = pd.read_sql_query(query,conn)
results.head()

,InvoiceDate,Track,Album,Artist,Genre,Media
0,2013-04-05 00:00:00,American Idiot,American Idiot,Green Day,Alternative & Punk,MPEG audio file
1,2013-04-05 00:00:00,Give Me Novacaine,American Idiot,Green Day,Alternative & Punk,MPEG audio file
2,2013-04-05 00:00:00,Whatsername,American Idiot,Green Day,Alternative & Punk,MPEG audio file
3,2013-04-05 00:00:00,Paradise City,Appetite for Destruction,Guns N' Roses,Rock,Protected AAC audio file
4,2013-04-05 00:00:00,Rocket Queen,Appetite for Destruction,Guns N' Roses,Rock,Protected AAC audio file


Q11. Produire la liste de tous les contacts de la base de données (clients & employés) avec leur nom, prénom, type (client ou employé), email, pays et no de téléphone


In [16]:
query = """SELECT c.FirstName, c.LastName, c.Email, c.Country, c.Phone, "Customer" as Type
FROM customers c
UNION
SELECT e.FirstName, e.LastName, e.Email, e.Country, e.Phone, "Employee" as Type
FROM employees e"""
results = pd.read_sql_query(query,conn)
results

,FirstName,LastName,Email,Country,Phone,Type
0,Aaron,Mitchell,aaronmitchell@yahoo.ca,Canada,+1 (204) 452-6452,Customer
1,Alexandre,Rocha,alero@uol.com.br,Brazil,+55 (11) 3055-3278,Customer
2,Andrew,Adams,andrew@chinookcorp.com,Canada,+1 (780) 428-9482,Employee
3,Astrid,Gruber,astrid.gruber@apple.at,Austria,+43 01 5134505,Customer
4,Bjørn,Hansen,bjorn.hansen@yahoo.no,Norway,+47 22 44 22 22,Customer
...,...,...,...,...,...,...
62,Steve,Murray,steve.murray@yahoo.uk,United Kingdom,+44 0131 315 3300,Customer
63,Terhi,Hämäläinen,terhi.hamalainen@apple.fi,Finland,+358 09 870 2000,Customer
64,Tim,Goyer,tgoyer@apple.com,USA,+1 (408) 996-1010,Customer
65,Victor,Stevens,vstevens@yahoo.com,USA,+1 (608) 257-0597,Customer


Q12. Produire la liste de répartition des ventes par pays (basée sur `BillingCountry`)

In [17]:
query = """SELECT BillingCountry, SUM(i.Total) AS Total, 
    ROUND((SUM(i.Total) / (SELECT SUM(i.Total) FROM invoices i) * 100),2) AS Percentage  
FROM invoices i
GROUP BY i.BillingCountry 
ORDER BY Total DESC"""
results = pd.read_sql_query(query,conn)
results.head()

,BillingCountry,Total,Percentage
0,USA,523.06,22.46
1,Canada,303.96,13.05
2,France,195.10,8.38
3,Brazil,190.10,8.16
4,Germany,156.48,6.72
